<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/%D0%BC%D0%BE%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_RAG_Retriaval_Augmented_Generation_%D0%B2%D0%B0%D1%80_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain

In [7]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
#Генерация эмбеддингов через sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

In [10]:
from typing import Any, Dict, List, Literal, Optional
import torch
import langchain
#from config import settings
#from langchain_chroma import Chroma
from langchain_deepseek import ChatDeepSeek
#from langchain_openai import ChatOpenAI
#from langchain_huggingface import HuggingFaceEmbeddings
from loguru import logger

ModuleNotFoundError: No module named 'langchain_deepseek'

In [9]:
#Класс интегрировации моделей DeepSeek и ChatGPT в систему
class ChatWithAI:
    def __init__(self, provider: Literal["deepseek", "openai"] = "deepseek"):
        self.provider = provider
        self.embeddings = HuggingFaceEmbeddings(
            model_name=settings.LM_MODEL_NAME,
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )

        if provider == "deepseek":
            self.llm = ChatDeepSeek(
                api_key=settings.DEEPSEEK_API_KEY,
                model=settings.DEEPSEEK_MODEL_NAME,
                temperature=0.7,
            )
        elif provider == "openai":
            self.llm = ChatOpenAI(
                api_key=settings.OPENAI_API_KEY,
                model=settings.OPENAI_MODEL_NAME,
                temperature=0.7,
            )
        else:
            raise ValueError(f"Неподдерживаемый провайдер: {provider}")

        self.chroma_db = Chroma(
            persist_directory=settings.AMVERA_CHROMA_PATH,
            embedding_function=self.embeddings,
            collection_name=settings.AMVERA_COLLECTION_NAME,
        )

In [ ]:
#Этот метод трансформирует связку документ + метаданные из всех полученных документов в одно структурированное сообщение, которое мы затем передадим нейросети

def format_context(self, context: List[Dict[str, Any]]) -> str:
    """Форматирование контекста для промпта."""
    formatted_context = []
    for item in context:
        metadata_str = "\n".join(f"{k}: {v}" for k, v in item["metadata"].items())
        formatted_context.append(
            f"Текст: {item['text']}\nМетаданные:\n{metadata_str}\n"
        )
    return "\n---\n".join(formatted_context)